In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from models import DCGAN

In [3]:
from preprocess_data import make_dataset, get_all_image_filenames

In [14]:
fnames = get_all_image_filenames()
N=len(fnames)
print("%d images found" % N)

6033 images found


In [15]:
epochs=50
batch_size=128

In [16]:
image_set = make_dataset(fnames)
tr_set = image_set.shuffle(N).repeat(epochs).batch(batch_size)

building dataset from 6033 images


In [6]:
noise_shape=(100,)
input_shape=(None, *noise_shape)

In [7]:
birdgan = DCGAN()
generator = birdgan.build_generator(input_shape)
discriminator = birdgan.build_discriminator()

Instructions for updating:
Colocations handled automatically by placer.


In [8]:
z = tf.random_uniform(noise_shape)


ValueError: Shape must be rank 1 but is rank 0 for 'random_uniform/RandomUniform' (op: 'RandomUniform') with input shapes: [].